In [45]:
import pandas as pd
import copy

df = pd.read_csv('buffy_pose_predictions.csv')

In [6]:
# pose prediction accuracy
accuracy = (df['label'] == df['predicted_pose']).sum()/len(df)
print("Accuracy of pose prediction:", accuracy)

Accuracy of pose prediction: 0.8421052631578947


In [39]:
# misclassified poses
pose_misclass = {"folded": {"folded": 0, "hips": 0, "rest": 0}, 
                 "hips": {"folded": 0, "hips": 0, "rest": 0},
                 "rest": {"folded": 0, "hips": 0, "rest": 0}}

for index, row in df.iterrows():
    if df['label'][index] != df['predicted_pose'][index]:
        pose_misclass[df.iloc[index]['label']][df.iloc[index]['predicted_pose']] += 1
        
print("Count of misclassified poses for each pose:")
for key in pose_misclass.keys():
    print(key + " pose: " + str(pose_misclass[key]))


Count of misclassified poses for each pose:
folded pose: {'folded': 0, 'hips': 0, 'rest': 0}
hips pose: {'folded': 0, 'hips': 0, 'rest': 1}
rest pose: {'folded': 0, 'hips': 2, 'rest': 0}


### Seems that "rest" and "hip" poses are most likely to be confused for each other.

In [54]:
# Determining if there exists a correlation between pose and emotion
emotions_dict = {'angry':0, 'sad':0, 'fear':0, 'neutral':0, 'happy':0}

# number of images with each emotion associated to each pose
pose_em_dict = {'folded': copy.deepcopy(emotions_dict),
               'hips': copy.deepcopy(emotions_dict),
               'rest': copy.deepcopy(emotions_dict)}
# normalized pose_em_dict by number of images per emotion
pose_em_norm = {'folded': copy.deepcopy(emotions_dict),
               'hips': copy.deepcopy(emotions_dict),
               'rest': copy.deepcopy(emotions_dict)}

# count number of images per emotion
num_emotions = copy.deepcopy(emotions_dict)
for row in range(len(df)):
    num_emotions[df.iloc[row]['predicted_emotion']] += 1

for index, row in df.iterrows():
    pose_em_dict[df.iloc[index]['label']][df.iloc[index]['predicted_emotion']] += 1
    pose_em_norm[df.iloc[index]['label']][df.iloc[index]['predicted_emotion']] += 1/num_emotions[df.iloc[index]['predicted_emotion']]

# convert dictionaries to df for easier display
pose_em_df = pd.DataFrame.from_dict(pose_em_dict, orient="index")
pose_em_norm_df = pd.DataFrame.from_dict(pose_em_norm, orient="index")
    
print('Number of images with each emotion associated to each pose:')
print(pose_em_df)
print()
    
print('Proportion of images with each emotion associated to each pose, normalized by number of images per emotion:')
print(pose_em_norm_df)

Number of images with each emotion associated to each pose:
        angry  sad  fear  neutral  happy
folded      0    0     1        0      1
hips        1    2     0        1      1
rest        1    5     4        2      0

Proportion of images with each emotion associated to each pose, normalized by number of images per emotion:
        angry       sad  fear   neutral  happy
folded    0.0  0.000000   0.2  0.000000    0.5
hips      0.5  0.285714   0.0  0.333333    0.5
rest      0.5  0.714286   0.8  0.666667    0.0
